# pyGPlates site reconstructions
Reconstruct paleolocations of present-day locations across the Phanerozoic. PyGplates allows to use some of the functionality of the GUI GPlates software within python scripts. This allows scripting to automatically process many different locations and/or time periods. Different rotation models can be used and are easily exchangeable. I currently use the PALEOMAP rotation model that is consistent to the Bristol Scotese simulations by Paul Valdes.

## User input
define variables/lists to quickly change inputs to the notebook

In [1]:
# input csv file with target age, modern latitude, modern longitude
csv_input = 'forcordall.csv'


## import pygplates and other packages

In [51]:
import os
import sys
import pandas as pd
import numpy as np
from tqdm import tqdm  # Import tqdm

sys.path.insert(0, os.path.abspath('./bin/pygplates_0.36.0_py37_Darwin-x86_64')) # macOS Arm
import pygplates


## load plate model
List of available models at http://portal.gplates.org/portal/rotation_models/.
Here we are using the 'PALEOMAP PaleoAtlas for GPlates'by Scotese et al. (https://www.earthbyte.org/paleomap-paleoatlas-for-gplates/)

In [10]:
# static polygons are the 'partitioning features'
static_polygons = pygplates.FeatureCollection('PALEOMAP_Global_Plate_Model/PALEOMAP_PlatePolygons.gpml')
# actual rotation model
rotation_model=pygplates.RotationModel('PALEOMAP_Global_Plate_Model/PALEOMAP_PlateModel.rot')

## Main code
3-step process to reconstruct paleolocations:
1. combine input points into feature collection
2. assign plate ids to feature
3. reconstruct paleolocations for feature

In [55]:
# load point coordinates
# df_sites = pd.read_csv('data/' + csv_input,sep=',', nrows = 1000)
df_sites = pd.read_csv('data/' + csv_input,sep=',')

reconstructed_lats = []
reconstructed_lons = []

# loop over each row and get paleolocation
for index, row in tqdm(df_sites.iterrows(), total=df_sites.shape[0], desc='Reconstructing paleolocations', unit='locations'):  # tqdm wrapper around the iterator
    mlat = float(df_sites['ModLat'].iloc[index])
    mlon = float(df_sites['ModLon'].iloc[index])
    age = float(df_sites['Age'].iloc[index])

    #### step 1: put the point into a feature collection, using Lat,Lon coordinates from dataframe
    point_features = []
    point = pygplates.PointOnSphere(mlat, mlon)
    point_feature = pygplates.Feature()
    point_feature.set_geometry(point)

    ### step 2: assign plate ids to feature
    # To reconstruct any feature geometries, each feature must have a plate id assigned. If they don't already, 
    # then the pygplates function 'PlatePartitioner' performs this function (analogous to the 'assign plate ids' 
    # menu option in GPlates GUI) 
    partitioned_point_feature = pygplates.partition_into_plates(static_polygons, rotation_model, point_feature) 

    ### step 3:Reconstruct the point features
    reconstructed_feature_geometry = []
    pygplates.reconstruct(partitioned_point_feature, rotation_model, reconstructed_feature_geometry, age)
    
    plat, plon = np.round(reconstructed_feature_geometry[0].get_reconstructed_geometry().to_lat_lon(),1)
    reconstructed_lats.append(plat)
    reconstructed_lons.append(plon)

    # # Each reconstructed geometry should be a point - return a list of all reconstructed points.
    # print(f'Coordinates of the reconstructed point with ModernLat: {mlat} / ModernLon: {mlon}', \
    #     reconstructed_feature_geometry[0].get_reconstructed_geometry().to_lat_lon() )

# Add the reconstructed data to the DataFrame
df_sites['PaleoLat'] = reconstructed_lats
df_sites['PaleoLon'] = reconstructed_lons

# Save the DataFrame to a new CSV file
df_sites.to_csv('data/forcordall_reconstructed.csv', index=False)

print("Reconstructed coordinates have been saved to 'forcordall_reconstructed.csv'")

Reconstructing paleolocations: 100%|██████████| 107685/107685 [29:58<00:00, 59.89locations/s] 


Reconstructed coordinates have been saved to 'forcordall_reconstructed.csv'
